In [ ]:
%matplotlib inline

import sys 
import os 

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 
from phdconf.config import *

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)

In [ ]:
index_name = 'filtered-phrasestop'#, config.SIGIR_INDEX_NAME]
qrel_paths =  [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['AUS']#, 'SIGIR']

In [ ]:
base_df = load_1d_dfs([index_name], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1)[0][0]
base_query = load_1d_dfs([index_name], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

In [ ]:
methods = ['idf', 'ictf', 'kli', 'plm']
start = 0 
end = 1.0
inc = 0.1 
_dir = os.path.join(BASE_DIR, 'reduction', 'dirichlet_prior')

#filtered-phrasestop-plm-1.00-topics-filtered-phrasestop-unigram_dir_mu_1050.00
method_runs = [load_1d_dfs([index_name], qrel_paths, _dir, '{0}-'+x+'-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, start, end, inc)[0] for x in methods]


In [ ]:
plot = plot_tune_1d_comp(methods, RERANK_METRICS, method_runs, start, end, inc, ylims=RERANK_YLIMS)
# plot.savefig('figures/ausnl-reduction.pdf')

In [ ]:
qry_runs = method_runs = [load_1d_dfs([index_name], qrel_paths, _dir, '{0}-'+x+'-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, start, end, inc, per_query=True)[0] for x in methods]

In [ ]:
(qry_runs[-1][-2]-base_query).sort_values('rbp@0.80')[-10:]

In [ ]:
def plot_len_correlation(queries, df, metrics):
    lens = {k: len(queries[k]['topic'].split()) for k in queries}
    df['lens'] = df.index.map(lens)
    
    fig = plt.figure() 
    ax = fig.add_subplot(111)
    fig.set_size_inches(16, 4)
    for m in metrics.keys():
        sns.regplot(x=df['lens'], y=df[m], ax=ax, truncate=False)
    ax.set_xlabel('Query length', fontsize=15)
    ax.set_ylabel('Effectiveness $\Delta$', fontsize=15)
    ax.set_ylim(-1, 1)
    ax.tick_params(labelsize=12)
    ax.legend(list(metrics.values()), frameon=True, fontsize=12).get_frame().set_edgecolor("black")
    
    return fig

# def load_queries(path: str):  
#     queries = {}
#     with open(path) as f:
#         data = json.load(f)
#         for topic in data['topics']:
#             leg_ref = None 
#             if 'legislation_ref' in topic: 
#                 leg_ref = topic['legislation_ref']
#             queries[topic['Id']] = {'topic': topic['Topic']}
#     return queries

    
red_dir = '/home/danlocke/go/src/github.com/dan-locke/phd/query-reduction'
# queries = load_queries(os.path.join(red_dir, 'filtered-phrasestop-ictf-0.80-topics.json'))
comp_df = load_1d_dfs([index_name], qrel_paths, _dir, '{0}-ictf-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, 0.80, 0.80, inc, per_query=True)[0][0]
interped_df = (comp_df - base_query)
len_corr_fig = plot_len_correlation(queries, interped_df, {'err@20': 'ERR@20', 'rbp@0.80': 'RBP'})
# len_corr_fig.savefig('figures/ausnl-len-reduction-8.pdf')

# queries = load_queries(os.path.join(red_dir, 'filtered-phrasestop-ictf-0.50-topics.json'))
comp_df = load_1d_dfs([index_name], qrel_paths, _dir, '{0}-ictf-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, 0.50, 0.50, inc, per_query=True)[0][0]
interped_df = (comp_df - base_query)
len_corr_fig = plot_len_correlation(queries, interped_df, {'err@20': 'ERR@20', 'rbp@0.80': 'RBP'})
# len_corr_fig.savefig('figures/ausnl-len-reduction-5.pdf')

In [ ]:
def correlation(queries, df, metrics):
    lens = {k: len(queries[k]['topic'].split()) for k in queries}
    df['lens'] = df.index.map(lens)
    out_df = pd.DataFrame()
    out_df['pearson'] = df[['lens', *list(metrics.keys())]].corr(method='pearson')['lens'].T
    out_df['kendall'] = df[['lens', *list(metrics.keys())]].corr(method='kendall')['lens'].T
    return out_df.drop(['lens'], axis='index').rename(metrics, axis='index').round(4).to_latex()
    
    
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
# write_table('tables/reduction-correlation', correlation(queries, interped_df, metrics))